In [2]:
import openai
import os
import requests
from dotenv import load_dotenv
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import json


load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

/Users/ernesto/Projects/Ideal/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
def get_workout(exercise_type, muscle_group, difficulty) -> str:
    url = "https://exercises-by-api-ninjas.p.rapidapi.com/v1/exercises"
    params = {
        "type": exercise_type,
        "muscle": muscle_group,
        "difficulty": difficulty,
    }
    headers = {

    }
    response = requests.get(url, headers = headers, params = params)
    if response.status_code == 200:
        return response.json()
    return response.status_code

In [4]:
def set_up(age, gender, height, current_weight, target_weight,fitness_goal, impediments):
    llm = ChatOpenAI(
        temperature = 0,
        model_name = "gpt-4",
    )
    prompt = """
             A {age} years old {gender} who currently weighs {current_weight} pounds, is {height} cm tall, and has the following impediments: {impediments},
             is trying to reach {target_weight} pounds and has the fitness goal: {fitness_goal}. Give a sample weekly workout plan.
             """
    prompt = PromptTemplate.from_template(prompt)
    chain = LLMChain(prompt = prompt, llm = llm, output_key = 'training_plan')
    
    prompt2 = """
              Take in a training plan: {training_plan}.
              For each day in the sample weekly workout plan, return the following in a list: type of exercise, muscle groups targeted, difficulty level.
              Make sure type of exercise is only one of the following: cardio, rest, strength. 
              Make sure the muscle groups targeted is in the following list: abdominals, adductors, biceps, calves, chest, forearms, glutes,
              hamstrings, lats, lower_back, middle_back, neck, quadriceps, traps, triceps.
              Make sure the diffculty level is one of the following: beginner, intermediate, expert.
              """
    prompt2 = PromptTemplate.from_template(prompt2)
    chain_two = LLMChain(llm = llm, prompt = prompt2, output_key = 'plan')

    prompt3 = """
              Take in a weekly workout plan: {plan}.
              Give it back in the format of a Python dictionary, where each day is a key. The value of each key is the information
              associated with it in the format of a python dictionary where exercise_type, muscle_group, and difficulty are the keys.
              Make sure to only include the dictionary in your response without including any other text.
              """
    prompt3 = PromptTemplate.from_template(prompt3)
    chain_three = LLMChain(llm = llm, prompt = prompt3)
    overall_chain = SequentialChain(
        chains = [chain, chain_two, chain_three],
        input_variables = ['age', 'gender', 'current_weight', 'height','impediments', 'target_weight', 'fitness_goal'],
        verbose = True
    )
    response = overall_chain(
        {'age': age,
         'gender' : gender,
         'current_weight' : current_weight  , 
         'height' : height,
         'impediments' : impediments,
         'target_weight' : target_weight,
         'fitness_goal' : fitness_goal}
    )
    return response

In [5]:
def get_results(age, gender, height, current_weight, target_weight,fitness_goal, impediments):
    text = json.loads(set_up(age, gender, height, current_weight, target_weight, fitness_goal, impediments)['text'])
    output = {}
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    for count, day in zip(weekdays, text.values()):
        if day['exercise_type'] == 'cardio':
            exercises = get_workout('cardio', "", day['difficulty'])
            temp = {}
            output[count] = exercises
        elif day['exercise_type'] == 'strength':
            temp = {}
            for group in day['muscle_group']:
                exercises = get_workout('strength', group, day['difficulty'])
                temp[group] = exercises
            output[count] = temp
        else:
            output[count] = 'Rest'
    return output

In [6]:
weekly_workout_plan = get_results(19, 'male', '6', '140', '160', 'trim fat', 'none')

/Users/ernesto/Projects/Ideal/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/ernesto/Projects/Ideal/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


In [7]:
weekly_workout_plan

{'Monday': {'chest': 401, 'triceps': 429},
 'Tuesday': 429,
 'Wednesday': 'Rest',
 'Thursday': {'lats': 429, 'biceps': 429},
 'Friday': 429,
 'Saturday': {'quadriceps': 429, 'glutes': 429, 'traps': 429},
 'Sunday': 'Rest'}

In [8]:
weekly_workout_plan.keys()

dict_keys(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

In [9]:
weekly_workout_plan['Monday'].keys()
weekly_workout_plan['Tuesday'].keys()
weekly_workout_plan['Wednesday'].keys()
weekly_workout_plan['Thursday'].keys()
weekly_workout_plan['Friday'].keys() 
weekly_workout_plan['Saturday'].keys()
weekly_workout_plan['Sunday'].keys()

AttributeError: 'int' object has no attribute 'keys'

In [81]:
weekly_workout_plan['Monday']['chest']

[{'name': 'Dumbbell Bench Press',
  'type': 'strength',
  'muscle': 'chest',
  'equipment': 'dumbbell',
  'difficulty': 'intermediate',
  'instructions': 'Lie down on a flat bench with a dumbbell in each hand resting on top of your thighs. The palms of your hands will be facing each other. Then, using your thighs to help raise the dumbbells up, lift the dumbbells one at a time so that you can hold them in front of you at shoulder width. Once at shoulder width, rotate your wrists forward so that the palms of your hands are facing away from you. The dumbbells should be just to the sides of your chest, with your upper arm and forearm creating a 90 degree angle. Be sure to maintain full control of the dumbbells at all times. This will be your starting position. Then, as you breathe out, use your chest to push the dumbbells up. Lock your arms at the top of the lift and squeeze your chest, hold for a second and then begin coming down slowly. Tip: Ideally, lowering the weight should take abou

In [82]:
weekly_workout_plan['Monday']['chest'][0]

{'name': 'Dumbbell Bench Press',
 'type': 'strength',
 'muscle': 'chest',
 'equipment': 'dumbbell',
 'difficulty': 'intermediate',
 'instructions': 'Lie down on a flat bench with a dumbbell in each hand resting on top of your thighs. The palms of your hands will be facing each other. Then, using your thighs to help raise the dumbbells up, lift the dumbbells one at a time so that you can hold them in front of you at shoulder width. Once at shoulder width, rotate your wrists forward so that the palms of your hands are facing away from you. The dumbbells should be just to the sides of your chest, with your upper arm and forearm creating a 90 degree angle. Be sure to maintain full control of the dumbbells at all times. This will be your starting position. Then, as you breathe out, use your chest to push the dumbbells up. Lock your arms at the top of the lift and squeeze your chest, hold for a second and then begin coming down slowly. Tip: Ideally, lowering the weight should take about twic